In [1]:
import os
import re
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch

%matplotlib inline

In [2]:
if torch.cuda.is_available():       
    device = torch.device("cuda:1")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 2 GPU(s) available.
Device name: GeForce RTX 2080 Ti


In [3]:
df_train = pd.read_csv('../Dataset/emotion/train.txt', header =None, sep =';', names = ['Input','Sentiment'], encoding='utf-8')
df_train

,Input,Sentiment
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger
...,...,...
15995,i just had a very brief time in the beanbag an...,sadness
15996,i am now turning and i feel pathetic that i am...,sadness
15997,i feel strong and good overall,joy
15998,i feel like this was such a rude comment and i...,anger


In [4]:
df_test = pd.read_csv('../Dataset/emotion/test.txt', header =None, sep =';', names = ['Input','Sentiment'], encoding='utf-8')
df_test

,Input,Sentiment
0,im feeling rather rotten so im not very ambiti...,sadness
1,im updating my blog because i feel shitty,sadness
2,i never make her separate from me because i do...,sadness
3,i left with my bouquet of red and yellow tulip...,joy
4,i was feeling a little vain when i did this one,sadness
...,...,...
1995,i just keep feeling like someone is being unki...,anger
1996,im feeling a little cranky negative after this...,anger
1997,i feel that i am useful to my people and that ...,joy
1998,im feeling more comfortable with derby i feel ...,joy


In [5]:
df_val = pd.read_csv('../Dataset/emotion/val.txt', header =None, sep =';', names = ['Input','Sentiment'], encoding='utf-8')
df_val

,Input,Sentiment
0,im feeling quite sad and sorry for myself but ...,sadness
1,i feel like i am still looking at a blank canv...,sadness
2,i feel like a faithful servant,love
3,i am just feeling cranky and blue,anger
4,i can have for a treat or if i am feeling festive,joy
...,...,...
1995,im having ssa examination tomorrow in the morn...,sadness
1996,i constantly worry about their fight against n...,joy
1997,i feel its important to share this info for th...,joy
1998,i truly feel that if you are passionate enough...,joy


In [6]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

encode = LabelEncoder()

labels_train = encode.fit_transform(df_train['Sentiment'])
labels_test = encode.fit_transform(df_test['Sentiment'])
labels_val = encode.fit_transform(df_val['Sentiment'])

y_train = np_utils.to_categorical(labels_train)
y_test = np_utils.to_categorical(labels_test)
y_val = np_utils.to_categorical(labels_val)

print(y_train.shape)
print(y_test.shape)
print(y_val.shape)

(16000, 6)
(2000, 6)
(2000, 6)


In [7]:
def text_preprocessing(text):
    """
    - Remove entity mentions (eg. '@united')
    - Correct errors (eg. '&amp;' to '&')
    @param    text (str): a string to be processed.
    @return   text (Str): the processed string.
    """
    # Remove '@name'
    text = re.sub(r'(@.*?)[\s]', ' ', text)

    # Replace '&amp;' with '&'
    text = re.sub(r'&amp;', '&', text)

    # Remove trailing whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [8]:
encoding = { 'anger': 0,
    'fear': 1,
    'joy': 2,
    'love': 3,
    'sadness': 4,
    'surprise': 5
}

X = np.concatenate([df_train['Input'].values, df_val['Input'].values])
X_train = df_train['Input'].values
y_train = [encoding[key] for key in df_train['Sentiment'].values]
X_val = df_val['Input'].values
y_val = [encoding[key] for key in df_val['Sentiment'].values]

In [9]:
print(y_train)

[4, 4, 0, 3, 0, 4, 5, 1, 2, 3, 4, 2, 0, 4, 2, 2, 4, 4, 4, 1, 0, 1, 2, 2, 0, 4, 4, 4, 0, 2, 2, 1, 5, 0, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 4, 4, 2, 3, 2, 0, 2, 4, 0, 1, 2, 4, 4, 5, 2, 2, 2, 3, 1, 1, 5, 0, 0, 4, 3, 2, 4, 4, 2, 4, 4, 4, 2, 2, 2, 0, 4, 0, 0, 0, 2, 2, 2, 2, 4, 1, 3, 0, 4, 0, 3, 4, 2, 2, 4, 0, 3, 2, 2, 2, 4, 2, 2, 2, 2, 4, 2, 2, 3, 4, 4, 2, 2, 4, 2, 2, 1, 1, 1, 4, 3, 2, 2, 3, 1, 5, 2, 2, 2, 2, 0, 1, 2, 0, 3, 0, 4, 2, 4, 0, 2, 5, 4, 0, 0, 4, 2, 1, 2, 2, 1, 4, 5, 5, 2, 0, 1, 0, 4, 0, 4, 1, 4, 2, 5, 1, 2, 0, 2, 0, 2, 1, 1, 4, 2, 2, 4, 5, 2, 1, 2, 4, 2, 2, 2, 1, 2, 5, 2, 0, 4, 4, 2, 0, 4, 2, 2, 5, 2, 1, 2, 1, 4, 1, 3, 4, 1, 3, 4, 4, 0, 2, 3, 0, 4, 5, 0, 2, 4, 0, 2, 2, 2, 2, 2, 4, 2, 3, 2, 4, 0, 5, 2, 0, 2, 3, 4, 4, 2, 4, 4, 2, 4, 2, 2, 2, 1, 4, 0, 4, 0, 3, 2, 3, 4, 2, 2, 2, 4, 2, 4, 0, 3, 4, 3, 4, 4, 4, 4, 4, 4, 1, 2, 4, 4, 2, 3, 4, 0, 4, 3, 2, 2, 2, 4, 2, 4, 2, 2, 2, 4, 4, 4, 3, 1, 4, 1, 2, 2, 1, 2, 0, 0, 3, 4, 5, 2, 0, 4, 4, 0, 3, 5, 4, 3, 2, 0, 2, 4, 4, 2, 2, 1, 4, 0, 2, 3, 2, 2, 

In [10]:
print(len(X))
print(len(X_train))
print(len(X_val))

18000
16000
2000


In [11]:
# Print sentence 0
print('Original: ', X[0])
print('Processed: ', text_preprocessing(X[0]))

Original:  i didnt feel humiliated
Processed:  i didnt feel humiliated


In [12]:
from transformers import BertTokenizer

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

# Create a function to tokenize a set of texts
def preprocessing_for_bert(data):
    """Perform required preprocessing steps for pretrained BERT.
    @param    data (np.array): Array of texts to be processed.
    @return   input_ids (torch.Tensor): Tensor of token ids to be fed to a model.
    @return   attention_masks (torch.Tensor): Tensor of indices specifying which
                  tokens should be attended to by the model.
    """
    # Create empty lists to store outputs
    input_ids = []
    attention_masks = []

    # For every sentence...
    for sent in data:
        # `encode_plus` will:
        #    (1) Tokenize the sentence
        #    (2) Add the `[CLS]` and `[SEP]` token to the start and end
        #    (3) Truncate/Pad sentence to max length
        #    (4) Map tokens to their IDs
        #    (5) Create attention mask
        #    (6) Return a dictionary of outputs
        encoded_sent = tokenizer.encode_plus(
            text=text_preprocessing(sent),  # Preprocess sentence
            add_special_tokens=True,        # Add `[CLS]` and `[SEP]`
            max_length=MAX_LEN,                  # Max length to truncate/pad
            pad_to_max_length=True,         # Pad sentence to max length
            #return_tensors='pt',           # Return PyTorch tensor
            return_attention_mask=True      # Return attention mask
            )
        
        # Add the outputs to the lists
        input_ids.append(encoded_sent.get('input_ids'))
        attention_masks.append(encoded_sent.get('attention_mask'))

    # Convert lists to tensors
    input_ids = torch.tensor(input_ids)
    attention_masks = torch.tensor(attention_masks)

    return input_ids, attention_masks

In [13]:
# Concatenate train data and test data
all_tweets = np.concatenate([df_train['Input'].values, df_test['Input'].values])

# Encode our concatenated data
encoded_tweets = [tokenizer.encode(sent, add_special_tokens=True) for sent in all_tweets]

# Find the maximum length
max_len = max([len(sent) for sent in encoded_tweets])
print('Max length: ', max_len)

Max length:  87


In [14]:
# Specify `MAX_LEN`
MAX_LEN = 87

# Print sentence 0 and its encoded token ids
token_ids = list(preprocessing_for_bert([X[0]])[0].squeeze().numpy())
print('Original: ', X[0])
print('Token IDs: ', token_ids)

# Run function `preprocessing_for_bert` on the train set and the validation set
print('Tokenizing data...')
train_inputs, train_masks = preprocessing_for_bert(X_train)
val_inputs, val_masks = preprocessing_for_bert(X_val)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/psimilan/anaconda3/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2136: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Original:  i didnt feel humiliated
Token IDs:  [101, 1045, 2134, 2102, 2514, 26608, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Tokenizing data...


In [15]:
print(train_inputs.shape)
print(train_masks.shape)

torch.Size([16000, 87])
torch.Size([16000, 87])


In [16]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# Convert other data types to torch.Tensor
train_labels = torch.tensor(y_train)
val_labels = torch.tensor(y_val)

# For fine-tuning BERT, the authors recommend a batch size of 16 or 32.
batch_size = 32

# Create the DataLoader for our training set
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Create the DataLoader for our validation set
val_data = TensorDataset(val_inputs, val_masks, val_labels)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)

In [46]:
print(train_data)

In [17]:
%%time
import torch
import torch.nn as nn
from transformers import BertModel

# Create the BertClassfier class
class BertClassifier(nn.Module):
    """Bert Model for Classification Tasks.
    """
    def __init__(self, freeze_bert=False):
        """
        @param    bert: a BertModel object
        @param    classifier: a torch.nn.Module classifier
        @param    freeze_bert (bool): Set `False` to fine-tune the BERT model
        """
        super(BertClassifier, self).__init__()
        # Specify hidden size of BERT, hidden size of our classifier, and number of labels
        D_in, H, D_out = 768, 50, 6

        # Instantiate BERT model
        self.bert = BertModel.from_pretrained('bert-base-uncased')

        # Instantiate an one-layer feed-forward classifier
        self.classifier = nn.Sequential(
            nn.Linear(D_in, H),
            nn.ReLU(),
            #nn.Dropout(0.5),
            nn.Linear(H, D_out)
        )

        # Freeze the BERT model
        if freeze_bert:
            for param in self.bert.parameters():
                param.requires_grad = False
        
    def forward(self, input_ids, attention_mask):
        """
        Feed input to BERT and the classifier to compute logits.
        @param    input_ids (torch.Tensor): an input tensor with shape (batch_size,
                      max_length)
        @param    attention_mask (torch.Tensor): a tensor that hold attention mask
                      information with shape (batch_size, max_length)
        @return   logits (torch.Tensor): an output tensor with shape (batch_size,
                      num_labels)
        """
        # Feed input to BERT
        outputs = self.bert(input_ids=input_ids,
                            attention_mask=attention_mask)
        
        # Extract the last hidden state of the token `[CLS]` for classification task
        last_hidden_state_cls = outputs[0][:, 0, :]

        # Feed input to classifier to compute logits
        logits = self.classifier(last_hidden_state_cls)

        return logits

CPU times: user 38 µs, sys: 0 ns, total: 38 µs
Wall time: 42.4 µs


In [18]:
from transformers import AdamW, get_linear_schedule_with_warmup

def initialize_model(epochs=10):
    """Initialize the Bert Classifier, the optimizer and the learning rate scheduler.
    """
    # Instantiate Bert Classifier
    bert_classifier = BertClassifier(freeze_bert=False)

    # Tell PyTorch to run the model on GPU
    bert_classifier.to(device)

    # Create the optimizer
    optimizer = AdamW(bert_classifier.parameters(),
                      lr=5e-5,    # Default learning rate
                      eps=1e-8    # Default epsilon value
                      )

    # Total number of training steps
    total_steps = len(train_dataloader) * epochs

    # Set up the learning rate scheduler
    scheduler = get_linear_schedule_with_warmup(optimizer,
                                                num_warmup_steps=0, # Default value
                                                num_training_steps=total_steps)
    return bert_classifier, optimizer, scheduler

In [19]:
import random
import time

# Specify loss function
loss_fn = nn.CrossEntropyLoss()

def set_seed(seed_value=42):
    """Set seed for reproducibility.
    """
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

def train(model, train_dataloader, val_dataloader=None, epochs=10, evaluation=False):
    """Train the BertClassifier model.
    """
    # Start training loop
    print("Start training...\n")
    for epoch_i in range(epochs):
        # =======================================
        #               Training
        # =======================================
        # Print the header of the result table
        print(f"{'Epoch':^7} | {'Batch':^7} | {'Train Loss':^12} | {'Val Loss':^10} | {'Val Acc':^9} | {'Elapsed':^9}")
        print("-"*70)

        # Measure the elapsed time of each epoch
        t0_epoch, t0_batch = time.time(), time.time()

        # Reset tracking variables at the beginning of each epoch
        total_loss, batch_loss, batch_counts = 0, 0, 0

        # Put the model into the training mode
        model.train()

        # For each batch of training data...
        for step, batch in enumerate(train_dataloader):
            batch_counts +=1
            # Load batch to GPU
            b_input_ids, b_attn_mask, b_labels = tuple(t.to(device) for t in batch)

            # Zero out any previously calculated gradients
            model.zero_grad()

            # Perform a forward pass. This will return logits.
            logits = model(b_input_ids, b_attn_mask)

            # Compute loss and accumulate the loss values
            #print(logits)
            #print(b_labels)
            loss = loss_fn(logits, b_labels)
            batch_loss += loss.item()
            total_loss += loss.item()

            # Perform a backward pass to calculate gradients
            loss.backward()

            # Clip the norm of the gradients to 1.0 to prevent "exploding gradients"
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # Update parameters and the learning rate
            optimizer.step()
            scheduler.step()

            # Print the loss values and time elapsed for every 20 batches
            if (step % 20 == 0 and step != 0) or (step == len(train_dataloader) - 1):
                # Calculate time elapsed for 20 batches
                time_elapsed = time.time() - t0_batch

                # Print training results
                print(f"{epoch_i + 1:^7} | {step:^7} | {batch_loss / batch_counts:^12.6f} | {'-':^10} | {'-':^9} | {time_elapsed:^9.2f}")

                # Reset batch tracking variables
                batch_loss, batch_counts = 0, 0
                t0_batch = time.time()

        # Calculate the average loss over the entire training data
        avg_train_loss = total_loss / len(train_dataloader)

        print("-"*70)
        # =======================================
        #               Evaluation
        # =======================================
        if evaluation == True:
            # After the completion of each training epoch, measure the model's performance
            # on our validation set.
            val_loss, val_accuracy = evaluate(model, val_dataloader)

            # Print performance over the entire training data
            time_elapsed = time.time() - t0_epoch
            
            print(f"{epoch_i + 1:^7} | {'-':^7} | {avg_train_loss:^12.6f} | {val_loss:^10.6f} | {val_accuracy:^9.2f} | {time_elapsed:^9.2f}")
            print("-"*70)
        print("\n")
    
    print("Training complete!")


def evaluate(model, val_dataloader):
    """After the completion of each training epoch, measure the model's performance
    on our validation set.
    """
    # Put the model into the evaluation mode. The dropout layers are disabled during
    # the test time.
    model.eval()

    # Tracking variables
    val_accuracy = []
    val_loss = []

    # For each batch in our validation set...
    for batch in val_dataloader:
        # Load batch to GPU
        b_input_ids, b_attn_mask, b_labels = tuple(t.to(device) for t in batch)

        # Compute logits
        with torch.no_grad():
            logits = model(b_input_ids, b_attn_mask)

        # Compute loss
        #print(logits)
        #print(b_labels)
        loss = loss_fn(logits, b_labels)
        val_loss.append(loss.item())

        # Get the predictions
        preds = torch.argmax(logits, dim=1).flatten()

        # Calculate the accuracy rate
        accuracy = (preds == b_labels).cpu().numpy().mean() * 100
        val_accuracy.append(accuracy)

    # Compute the average accuracy and loss over the validation set.
    val_loss = np.mean(val_loss)
    val_accuracy = np.mean(val_accuracy)

    return val_loss, val_accuracy

In [20]:
set_seed(42)    # Set seed for reproducibility
bert_classifier, optimizer, scheduler = initialize_model(epochs=10)
train(bert_classifier, train_dataloader, val_dataloader, epochs=10, evaluation=True)

Start training...

 Epoch  |  Batch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
----------------------------------------------------------------------
   1    |   20    |   1.572703   |     -      |     -     |   3.71   
   1    |   40    |   1.309237   |     -      |     -     |   3.51   
   1    |   60    |   1.063842   |     -      |     -     |   3.53   
   1    |   80    |   0.882139   |     -      |     -     |   3.54   
   1    |   100   |   0.703019   |     -      |     -     |   3.54   
   1    |   120   |   0.490705   |     -      |     -     |   3.52   
   1    |   140   |   0.423874   |     -      |     -     |   3.54   
   1    |   160   |   0.412985   |     -      |     -     |   3.55   
   1    |   180   |   0.337185   |     -      |     -     |   3.58   
   1    |   200   |   0.291516   |     -      |     -     |   3.59   
   1    |   220   |   0.325266   |     -      |     -     |   3.56   
   1    |   240   |   0.292858   |     -      |     -     |   3.60   


   4    |    -    |   0.087427   |  0.173352  |   93.75   |   94.16  
----------------------------------------------------------------------


 Epoch  |  Batch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
----------------------------------------------------------------------
   5    |   20    |   0.059183   |     -      |     -     |   3.79   
   5    |   40    |   0.059305   |     -      |     -     |   3.63   
   5    |   60    |   0.076254   |     -      |     -     |   3.62   
   5    |   80    |   0.079035   |     -      |     -     |   3.61   
   5    |   100   |   0.051603   |     -      |     -     |   3.62   
   5    |   120   |   0.050878   |     -      |     -     |   3.62   
   5    |   140   |   0.044038   |     -      |     -     |   3.61   
   5    |   160   |   0.055733   |     -      |     -     |   3.64   
   5    |   180   |   0.056302   |     -      |     -     |   3.62   
   5    |   200   |   0.083764   |     -      |     -     |   3.63   
   5    |   220 

   8    |   480   |   0.006229   |     -      |     -     |   3.61   
   8    |   499   |   0.016093   |     -      |     -     |   3.45   
----------------------------------------------------------------------
   8    |    -    |   0.013070   |  0.301034  |   94.05   |   95.74  
----------------------------------------------------------------------


 Epoch  |  Batch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
----------------------------------------------------------------------
   9    |   20    |   0.024105   |     -      |     -     |   3.79   
   9    |   40    |   0.001508   |     -      |     -     |   3.60   
   9    |   60    |   0.001022   |     -      |     -     |   3.60   
   9    |   80    |   0.009984   |     -      |     -     |   3.61   
   9    |   100   |   0.007413   |     -      |     -     |   3.61   
   9    |   120   |   0.021758   |     -      |     -     |   3.61   
   9    |   140   |   0.010834   |     -      |     -     |   3.61   
   9    |   160

In [27]:
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
print(input)
print(target)

tensor([[-0.4425, -1.2383,  0.5189, -0.4659, -0.6504],
        [ 0.3995,  0.0748, -0.3477, -2.0204, -1.4760],
        [-0.7025,  0.5460, -0.2742,  1.1410, -1.3207]], requires_grad=True)
tensor([3, 4, 1])


In [21]:
from sklearn.metrics import accuracy_score, roc_curve, auc

def evaluate_roc(probs, y_true):
    """
    - Print AUC and accuracy on the test set
    - Plot ROC
    @params    probs (np.array): an array of predicted probabilities with shape (len(y_true), 2)
    @params    y_true (np.array): an array of the true values with shape (len(y_true),)
    """
    preds = probs[:, 1]
    fpr, tpr, threshold = roc_curve(y_true, preds)
    roc_auc = auc(fpr, tpr)
    print(f'AUC: {roc_auc:.4f}')
       
    # Get accuracy over the test set
    y_pred = np.where(preds >= 0.5, 1, 0)
    accuracy = accuracy_score(y_true, y_pred)
    print(f'Accuracy: {accuracy*100:.2f}%')
    
    # Plot ROC AUC
    plt.title('Receiver Operating Characteristic')
    plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()

In [22]:
import torch.nn.functional as F

def bert_predict(model, test_dataloader):
    """Perform a forward pass on the trained BERT model to predict probabilities
    on the test set.
    """
    # Put the model into the evaluation mode. The dropout layers are disabled during
    # the test time.
    model.eval()

    all_logits = []

    # For each batch in our test set...
    for batch in test_dataloader:
        # Load batch to GPU
        b_input_ids, b_attn_mask = tuple(t.to(device) for t in batch)[:2]

        # Compute logits
        with torch.no_grad():
            logits = model(b_input_ids, b_attn_mask)
        all_logits.append(logits)
    
    # Concatenate logits from each batch
    all_logits = torch.cat(all_logits, dim=0)

    # Apply softmax to calculate probabilities
    probs = F.softmax(all_logits, dim=1).cpu().numpy()

    return probs

In [55]:
# Compute predicted probabilities on the test set
probs = bert_predict(bert_classifier, val_dataloader)

# Evaluate the Bert classifier
evaluate_roc(probs, y_val)

ValueError: multiclass format is not supported

In [36]:
# Concatenate the train set and the validation set
full_train_data = torch.utils.data.ConcatDataset([train_data, val_data])
full_train_sampler = RandomSampler(full_train_data)
full_train_dataloader = DataLoader(full_train_data, sampler=full_train_sampler, batch_size=32)

# Train the Bert Classifier on the entire training data
set_seed(42)
bert_classifier, optimizer, scheduler = initialize_model(epochs=2)
train(bert_classifier, full_train_dataloader, epochs=2)

Start training...

 Epoch  |  Batch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
----------------------------------------------------------------------
   1    |   20    |   0.395516   |     -      |     -     |   2.92   
   1    |   40    |   0.076552   |     -      |     -     |   2.74   
   1    |   60    |   0.041534   |     -      |     -     |   2.72   
   1    |   80    |   0.056735   |     -      |     -     |   2.75   
   1    |   100   |   0.047529   |     -      |     -     |   2.70   
   1    |   120   |   0.029075   |     -      |     -     |   2.72   
   1    |   140   |   0.017800   |     -      |     -     |   2.71   
   1    |   160   |   0.021259   |     -      |     -     |   2.71   
   1    |   180   |   0.023377   |     -      |     -     |   2.70   
   1    |   200   |   0.035580   |     -      |     -     |   2.69   
   1    |   220   |   0.025894   |     -      |     -     |   2.70   
   1    |   240   |   0.041768   |     -      |     -     |   2.72   


In [23]:
df_test.sample(5)

,Input,Sentiment
1860,i feel so dirty but after spending a day at th...,sadness
353,i could feel his breath on me and smell the sw...,joy
1333,i just want to feel loved by you,love
905,i have felt the need to write out my sometimes...,anger
1289,at a party i met a girl who drew me to her,anger


In [24]:
# Run `preprocessing_for_bert` on the test set
print('Tokenizing data...')
test_inputs, test_masks = preprocessing_for_bert(df_test['Input'].values)

# Create the DataLoader for our test set
test_dataset = TensorDataset(test_inputs, test_masks)
test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=32)

Tokenizing data...


/home/psimilan/anaconda3/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2136: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [25]:
# Compute predicted probabilities on the test set
probs = bert_predict(bert_classifier, test_dataloader)

# Get predictions from the probabilities
threshold = 0.000135
preds = np.where(probs[:, 1] > threshold, 1, 0)

# Number of tweets predicted non-negative
print("Number of tweets predicted non-negative: ", preds)

Number of tweets predicted non-negative:  [0 0 0 ... 0 0 1]


In [26]:
from collections import Counter

print("Balanced Checking: ", Counter(df_test['Sentiment']))

Balanced Checking:  Counter({'joy': 695, 'sadness': 581, 'anger': 275, 'fear': 224, 'love': 159, 'surprise': 66})


In [27]:
y_test = [encoding[key] for key in df_test['Sentiment'].values]
labels = np.argmax(probs, axis=-1)
print(probs)
print(labels)
print(y_test)

[[6.61799131e-05 3.20299005e-05 5.06104443e-05 3.41443920e-05
  9.99780715e-01 3.63394502e-05]
 [6.03461631e-05 3.68089532e-05 2.72166490e-05 3.08095223e-05
  9.99802530e-01 4.23386555e-05]
 [6.10930656e-05 3.28891365e-05 2.76123556e-05 3.32480813e-05
  9.99805987e-01 3.91205976e-05]
 ...
 [4.14003480e-05 3.83911502e-05 9.99844551e-01 4.57402093e-05
  1.00497555e-05 1.98572088e-05]
 [4.22703451e-05 4.98949703e-05 9.99835730e-01 4.18460259e-05
  1.15720168e-05 1.86609686e-05]
 [2.63132178e-03 8.87321591e-01 1.28139579e-03 3.57865880e-04
  9.94306174e-04 1.07413530e-01]]
[4 4 4 ... 2 2 1]
[4, 4, 4, 2, 4, 1, 0, 2, 2, 0, 1, 4, 1, 2, 3, 4, 2, 4, 0, 2, 4, 2, 2, 4, 4, 1, 0, 4, 1, 0, 1, 0, 4, 0, 4, 2, 2, 4, 2, 2, 0, 4, 2, 4, 2, 0, 2, 2, 1, 1, 4, 1, 2, 4, 2, 4, 4, 2, 4, 0, 4, 4, 2, 2, 4, 5, 4, 0, 1, 5, 2, 3, 5, 2, 3, 0, 2, 4, 2, 3, 2, 0, 4, 2, 4, 4, 2, 2, 2, 4, 2, 1, 0, 1, 0, 0, 3, 4, 0, 4, 4, 4, 4, 1, 0, 0, 2, 2, 1, 4, 2, 3, 1, 2, 4, 2, 2, 1, 4, 2, 4, 2, 4, 0, 4, 3, 4, 1, 4, 4, 2, 3, 4, 0, 0, 

In [28]:
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

y_test = [encoding[key] for key in df_test['Sentiment'].values]
preds = np.argmax(probs, axis=-1)
accuracy = accuracy_score(y_test, preds)
f1 = f1_score(y_test, preds, average='weighted')
print("Accuracy: %.2f%%" % (accuracy*100))
print("F1 Score: %.2f" % (f1*100))

print(classification_report(y_test, preds))

Accuracy: 92.85%
F1 Score: 92.81
              precision    recall  f1-score   support

           0       0.93      0.91      0.92       275
           1       0.88      0.91      0.89       224
           2       0.94      0.95      0.95       695
           3       0.82      0.81      0.81       159
           4       0.97      0.97      0.97       581
           5       0.81      0.71      0.76        66

    accuracy                           0.93      2000
   macro avg       0.89      0.88      0.88      2000
weighted avg       0.93      0.93      0.93      2000



In [29]:
import pickle
with open('./model/Fine-Tuning_BERT-Emotion.h5', 'wb') as model:
    pickle.dump(bert_classifier, model)